<a href="https://colab.research.google.com/github/querzy/hello-world/blob/master/Model_2_LSTM_PREDICTIONS_EURUSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

REINITIALISER L'ENVIRONNEMNT D'EXECUTION !!!
- Execution - reinitialiser l'environnement d'execution
- Executer le script plusieurs fois
- Utiliser le dossier Oanda currency download2 (4000 barres)
- Retirer une barre du fichier si barre ouverte et considérer le resultat à t0 OU considérer le résultat à t-1 avec une barre ouverte.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving EURUSD1440.csv to EURUSD1440.csv


In [2]:
#@title
import numpy as np
import pandas as pd
#import datetime as datetime
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
 
import tensorflow as tf
from tensorflow import keras
 
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")
#import chart_studio.plotly as py
#import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
#pyoff.init_notebook_mode(connected=True)
from numpy.random import seed
seed(1)
tf.random.set_seed(2) 
 
 
#def parser(x):
#    return pd.datetime.strptime('190'+x, '%Y-%m')
 
 
#start='2020.08.21'
#rng=pd.date_range(start, periods=5, freq='B').strftime('%Y-%m-%d 21:00:00')
#df = pd.DataFrame({ 'date': rng, 'qty': np.random.randn(len(rng)) }) 
 
#plot daily prices
#plot_data = [
#    go.Scatter(
#        x=df['date'],
#        y=df['qty'],
#    )
#]
 
#plot_layout = go.Layout(
#        title='Daily Price'
#    )
#fig = go.Figure(data=plot_data, layout=plot_layout)
#fig.show()
 
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)
 
file='EURUSD1440.csv'
 
keepProgramRunning = True
while keepProgramRunning:
    print( "\n\n                              Sélectionner le TIME FRAME : \n")
    print( "          0: TF Montly \n")
    print( "          1: TF Weekly \n")
    print( "          2: TF Daily \n")
    print( "          3: TF 480 \n")
    print( "          4: TF 240 \n")
    print( "          5: TF 120 \n")
    print( "          6: TF 60 \n")
    print( "          7: TF 30 \n")
    print( "          8: TF 15 \n")
    print( "          9: TF 5 \n")
    print( "          10: TF 1 \n")
              
    choix = input()
    if choix =="0" :
        TimeFrame = 'Monthly'
        epochs=int(6)
        keepProgramRunning = False
    elif choix =="1" :
        TimeFrame = 'Weekly'
        epochs=int(6)
        keepProgramRunning = False
    elif choix =="2" :
        TimeFrame = 'Daily'
        epochs=int(5)#EURUSD
        #epochs=int(20)#BTCUSD
        keepProgramRunning = False
    elif choix =="3" :
        TimeFrame = '8H'
        epochs=int(6)   
        keepProgramRunning = False
    elif choix =="4" :
        TimeFrame = '4H'
        epochs=int(3)
        keepProgramRunning = False
    elif choix =="5" :
        TimeFrame = '2H'
        epochs=int(7)   
        keepProgramRunning = False
    elif choix =="6" :
        TimeFrame = '1H'
        epochs=int(6)
        keepProgramRunning = False
    elif choix =="7" :
        TimeFrame = '30 min'
        epochs=int(6)
        keepProgramRunning = False
    elif choix =="8" :
        TimeFrame = '15 min'
        epochs=int(5)
        keepProgramRunning = False
    elif choix =="9" :
        TimeFrame = '5 min'
        epochs=int(6)
        keepProgramRunning = False
    elif choix =="10" :
        TimeFrame = '1 min'
        epochs=int(6)
        keepProgramRunning = False
 
t0up=0
t0dn=0 
PRICEAVt0=[]
for j in range(2):
  df = pd.read_csv(file, index_col='Date',usecols=['Date', 'Close'],date_parser = pd.to_datetime)
  df.tail()
 
  print(df)
 
  train = df
 
  scaler = MinMaxScaler()
  scaler.fit(train)
  train = scaler.transform(train)
 
  n_input = 12
  n_features = 1
  generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
 
  model = Sequential()
  #model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
  model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
  #model.add(Dropout(0.15))
  model.add(Dense(1))
 
  #Model summary
  #model.summary()
  #Compiling
  #model.compile(optimizer='adam', loss = 'mse')
 
  optimizer = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer, loss='mse')
 
  history = model.fit(generator,epochs=5,verbose=1)#EURUSD
  #history = model.fit(generator,epochs=20,verbose=1)#BTCUSD
 
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
 
  plot_data = [
      go.Scatter(
          x=hist['epoch'],
          y=hist['loss'],
          name='loss'
      )
      
  ]
 
  plot_layout = go.Layout(
          title='Training loss'
      )
  fig = go.Figure(data=plot_data, layout=plot_layout)
  #fig.show()
  pyoff.iplot(fig)
 
  pred_list = []
 
  batch = train[-n_input:].reshape((1, n_input, n_features))
 
  for i in range(n_input):   
      pred_list.append(model.predict(batch)[0]) 
      batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
 
  if choix =="1" :
    add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
  elif choix =="2" :
    add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
  elif choix =="4" :
    add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]  
  elif choix =="6" :
    add_dates = [df.index[-1] + DateOffset(hours=x) for x in range(0,13) ] 
  #add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
  #add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
  #add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]
  
  future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)
  print(add_dates)
  df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                            index=future_dates[-n_input:].index, columns=['Prediction'])
 
  df_proj = pd.concat([df,df_predict], axis=1)
 
  df_proj.tail(12)
 
  plot_data = [
      go.Scatter(
          x=df_proj.index[-30:],
          y=df_proj['Close'][-30:],
          name='actual'
      ),
      go.Scatter(
          x=df_proj.index[-30:],
          y=df_proj['Prediction'][-30:],
          name='prediction'
      )
  ]
 
  plot_layout = go.Layout(
          title=file+' prediction t0'
      )
 
  fig = go.Figure(data=plot_data, layout=plot_layout)
  #fig.show()
  pyoff.iplot(fig)
  
  
  yt0Cl=df['Close'].iloc[-1]
  yt0Pr=df_proj['Prediction'].iloc[-12]
  PRICEAVt0.append(yt0Pr)
  print(yt0Cl)
  print(yt0Pr)
  t0diff=yt0Cl-yt0Pr
  if t0diff>0:
    t0dn=t0dn+1
  elif t0diff<=0 :
    t0up=t0up+1
      
prixmoyt0=mean(PRICEAVt0)
  #########################################
  #########################################
 
t1up=0
t1dn=0 
PRICEAVt1=[]
for j in range(20):
  df = pd.read_csv(file, skipfooter=1, engine='python', index_col='Date',usecols=['Date', 'Close'],date_parser = pd.to_datetime)
  df.tail()
 
  print(df)
 
  train = df
 
  scaler = MinMaxScaler()
  scaler.fit(train)
  train = scaler.transform(train)
 
  n_input = 12
  n_features = 1
  generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
 
  model = Sequential()
  #model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
  model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
  #model.add(Dropout(0.15))
  model.add(Dense(1))
 
  #Model summary
  #model.summary()
  #Compiling
  #model.compile(optimizer='adam', loss = 'mse')
 
  optimizer = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer, loss='mse')
 
  history = model.fit(generator,epochs=5,verbose=1)#EURUSD
  #history = model.fit(generator,epochs=20,verbose=1)#BTCUSD
 
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
 
  plot_data = [
      go.Scatter(
          x=hist['epoch'],
          y=hist['loss'],
          name='loss'
      )
      
  ]
 
  plot_layout = go.Layout(
          title='Training loss'
      )
  fig = go.Figure(data=plot_data, layout=plot_layout)
  #fig.show()
  pyoff.iplot(fig)
 
  pred_list = []
 
  batch = train[-n_input:].reshape((1, n_input, n_features))
 
  for i in range(n_input):   
      pred_list.append(model.predict(batch)[0]) 
      batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
 
  if choix =="1" :
    add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
  elif choix =="2" :
    add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
  elif choix =="4" :
    add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]  
  
  #add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
  #add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
  #add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]
  
  future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)
  print(add_dates)
  df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                            index=future_dates[-n_input:].index, columns=['Prediction'])
 
  df_proj = pd.concat([df,df_predict], axis=1)
 
  df_proj.tail(12)
 
  plot_data = [
      go.Scatter(
          x=df_proj.index[-30:],
          y=df_proj['Close'][-30:],
          name='actual'
      ),
      go.Scatter(
          x=df_proj.index[-30:],
          y=df_proj['Prediction'][-30:],
          name='prediction'
      )
  ]
 
  plot_layout = go.Layout(
          title=file+' prediction t-1'
      )
  fig = go.Figure(data=plot_data, layout=plot_layout)
  #fig.show()
  pyoff.iplot(fig)
 
  yt1Cl=df['Close'].iloc[-1]
  yt1Pr=df_proj['Prediction'].iloc[-12]
  PRICEAVt1.append(yt1Pr)
  print(yt1Cl)
  print(yt1Pr)
  t1diff=yt1Cl-yt1Pr
  if t1diff>0:
    t1dn=t1dn+1
  elif t1diff<=0:
    t1up=t1up+1
 
prixmoyt1=mean(PRICEAVt1) 
print(' ')
print('t0up : ',t0up)
print('t0dn : ',t0dn)
print('prix moyen t0 : ',prixmoyt0) 
print(' ')
print('t1up : ',t1up)
print('t1dn : ',t1dn)
print('prix moyen t1 : ',prixmoyt1)



                              Sélectionner le TIME FRAME : 

          0: TF Montly 

          1: TF Weekly 

          2: TF Daily 

          3: TF 480 

          4: TF 240 

          5: TF 120 

          6: TF 60 

          7: TF 30 

          8: TF 15 

          9: TF 5 

          10: TF 1 

2
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298
2021-02-11 22:00:00  1.21223

[3999 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 6s 7ms/step - loss: 0.0118
Epoch 2/5
665/665 [==============================] - 5s 7ms/step - loss: 0.0010
Epoch 3/5
665/665 [==============================] - 5s 7ms/step - loss: 6.8042e-04
Epoch 4/5
665/665 [==========================

[Timestamp('2021-02-11 22:00:00'), Timestamp('2021-02-12 22:00:00'), Timestamp('2021-02-13 22:00:00'), Timestamp('2021-02-14 22:00:00'), Timestamp('2021-02-15 22:00:00'), Timestamp('2021-02-16 22:00:00'), Timestamp('2021-02-17 22:00:00'), Timestamp('2021-02-18 22:00:00'), Timestamp('2021-02-19 22:00:00'), Timestamp('2021-02-20 22:00:00'), Timestamp('2021-02-21 22:00:00'), Timestamp('2021-02-22 22:00:00'), Timestamp('2021-02-23 22:00:00')]


1.21223
1.2142163030409816
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298
2021-02-11 22:00:00  1.21223

[3999 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 6s 7ms/step - loss: 0.0120
Epoch 2/5
665/665 [==============================] - 5s 7ms/step - loss: 7.3932e-04
Epoch 3/5
665/665 [==============================] - 5s 7ms/step - loss: 5.4638e-04
Epoch 4/5
665/665 [==============================] - 5s 7ms/step - loss: 5.4146e-04
Epoch 5/5
665/665 [==============================] - 5s 7ms/step - loss: 5.0610e-04


[Timestamp('2021-02-11 22:00:00'), Timestamp('2021-02-12 22:00:00'), Timestamp('2021-02-13 22:00:00'), Timestamp('2021-02-14 22:00:00'), Timestamp('2021-02-15 22:00:00'), Timestamp('2021-02-16 22:00:00'), Timestamp('2021-02-17 22:00:00'), Timestamp('2021-02-18 22:00:00'), Timestamp('2021-02-19 22:00:00'), Timestamp('2021-02-20 22:00:00'), Timestamp('2021-02-21 22:00:00'), Timestamp('2021-02-22 22:00:00'), Timestamp('2021-02-23 22:00:00')]


1.21223
1.2092155512684586
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-04 22:00:00  1.20466
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298

[3998 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 6s 7ms/step - loss: 0.0190
Epoch 2/5
665/665 [==============================] - 5s 7ms/step - loss: 8.0815e-04
Epoch 3/5
665/665 [==============================] - 5s 7ms/step - loss: 6.4144e-04
Epoch 4/5
665/665 [==============================] - 5s 7ms/step - loss: 6.5719e-04
Epoch 5/5
665/665 [==============================] - 5s 7ms/step - loss: 5.1355e-04


...

1.21298
1.2112909379893544
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-04 22:00:00  1.20466
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298

[3998 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 7s 8ms/step - loss: 0.0221
Epoch 2/5
665/665 [==============================] - 6s 8ms/step - loss: 7.6523e-04
Epoch 3/5
665/665 [==============================] - 6s 8ms/step - loss: 6.0629e-04
Epoch 4/5
665/665 [==============================] - 6s 8ms/step - loss: 5.2619e-04
Epoch 5/5
665/665 [==============================] - 6s 8ms/step - loss: 5.1416e-04


[Timestamp('2021-02-10 22:00:00'), Timestamp('2021-02-11 22:00:00'), Timestamp('2021-02-12 22:00:00'), Timestamp('2021-02-13 22:00:00'), Timestamp('2021-02-14 22:00:00'), Timestamp('2021-02-15 22:00:00'), Timestamp('2021-02-16 22:00:00'), Timestamp('2021-02-17 22:00:00'), Timestamp('2021-02-18 22:00:00'), Timestamp('2021-02-19 22:00:00'), Timestamp('2021-02-20 22:00:00'), Timestamp('2021-02-21 22:00:00'), Timestamp('2021-02-22 22:00:00')]


1.21298
1.2168541000235082
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-04 22:00:00  1.20466
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298

[3998 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 7s 8ms/step - loss: 0.0234
Epoch 2/5
665/665 [==============================] - 6s 8ms/step - loss: 0.0011
Epoch 3/5
665/665 [==============================] - 6s 9ms/step - loss: 7.8772e-04
Epoch 4/5
665/665 [==============================] - 6s 9ms/step - loss: 6.2504e-04
Epoch 5/5
665/665 [==============================] - 6s 8ms/step - loss: 5.5196e-04


[Timestamp('2021-02-10 22:00:00'), Timestamp('2021-02-11 22:00:00'), Timestamp('2021-02-12 22:00:00'), Timestamp('2021-02-13 22:00:00'), Timestamp('2021-02-14 22:00:00'), Timestamp('2021-02-15 22:00:00'), Timestamp('2021-02-16 22:00:00'), Timestamp('2021-02-17 22:00:00'), Timestamp('2021-02-18 22:00:00'), Timestamp('2021-02-19 22:00:00'), Timestamp('2021-02-20 22:00:00'), Timestamp('2021-02-21 22:00:00'), Timestamp('2021-02-22 22:00:00')]


1.21298
1.2051008010756972
 
t0up :  1
t0dn :  1
prix moyen t0 :  1.21171592715472
 
t1up :  4
t1dn :  16
prix moyen t1 :  1.2090683319152595
